# Database

## 参考文献

* [MySQL事务隔离级别](https://developer.aliyun.com/article/743691)
* [深入迁出MySQL与InnoDB](https://draveness.me/mysql-innodb/)
* [为什么MySQL使用B+树](https://draveness.me/whys-the-design-mysql-b-plus-tree/)
* [一分钟明白MySQL聚簇索引和非聚簇索引](https://www.cnblogs.com/sy270321/p/12864357.html)

## MySQL架构

### MySQL逻辑架构

* 连接管理、认证授权
* 查询解析、优化、缓存， 存储过程、触发器、视图等高级应用
* 存储引擎

### 并发控制

读写锁

* 读锁read lock，通常也称作共享锁shared lock
* 写锁write lock，通常也称作排他所exclusive lock

锁的粒度

* 表锁table lock
* 行锁row lock

### Transaction

数据库事务（Database Transaction）是所有的操作是一个原子操作，要么全部执行，要么不执行。

谈到事务，就要涉及到ACID，分别表示原子性(Atomicity)、一致性(Consistency)、隔离性(Isolation)和持久性(Durability)。

* 原子性Atomicity, 一个事务可以看作一个原子操作，要不全部执行，要不失败回滚
* 一致性Consistency，数据库总是从一个一致性状态转移到另一个一致性状态
* 隔离性Isolation，一个事务提交以前，对其他事务是不可见的
* 持久性Durability，一旦事务提交了，就会永久保存到数据库中

#### 隔离级别

SQL标准中定义了四种隔离级别，每种隔离级别都规定了一个事务中所做的修改，哪些修改在事务内和事务间可见，哪些是不可见的。

* READ UNCOMMITED 未提交读
* READ COMMITTED 提交读
* REPEATED READ 可重复读
* SERIALIZABLE 可串行化

**READ UNCOMMITED 未提交读**

事务的修改，即使没有提交，对其他事务也都是可见的。事务可以读未提交的数据，这也成为脏读Dirty Read.

**READ COMMITTED 提交读**

大多数数据库的默认隔离级别时READ COMMITED（但MySQL不是）。READ COMMITTED满足前面的提到的隔离性的定义：一个事务开始时，只能看见已经提交事务所做的修改。换句话说，一个事务从开始直到提交之前，所做的修改对其他事务都是不可见的。这个级别有时候也叫做不可重复读，因为两次执行相同的查询，可能会得到不一样的结果。

**REPEATED READ 可重复读**

REPEATED READ解决了脏读的问题。该级别保证了在同一个事物中多次读取同样的记录结果上一致的。但是理论上，可重复读隔离级别还是无法解决另外一个幻读的问题。所谓幻读，指的是当事务在读取某个范围内的记录时，另外一个事务又在这个范围内插入了新的记录，当之前的事务再次读取该范围的记录时，会产生幻行(Phantom ROw)。InnoDB存储引擎通过多版本控制解决了幻读的问题。
**可重复读是MySQL的默认隔离级别**

**SERIALIZABLE 可串行化**

它通过强制事务串行执行，避免了前面说的幻读的问题。简单来说，SERIALIZABLE会在读取的每一行数据上都加锁，所可能导致大量的超时和锁竞争的问题。实际应用各种很少用到这个隔离级别。

#### 死锁dead lock

死锁是指两个或多个事务在同一个资源上互相占用，并请求锁定对方占用的资源，从而导致恶性循环的线性。

为了解决这个问题，数据库实现了各种死锁检测和死锁超时机制。例如InnoDB存储引擎，就能检测死锁循环，并立即返回一个错误。目前，InnoDB处理方式是，将持有最少行的排他锁的事务进行回滚。

### 多版本并发控制MVCC

MVCC是Multi-Version Concurrent Control的简称。

多版本控制是行锁的一个变种，在很多情况下避免了加锁操作，因此开销更低。

InnoDB的MVCC，是通过在每行记录后面保存两个隐藏的列来实现的。这两列分别保存了行的创建时间和过期时间（删除时间）。当然存储的不是实际时间，而是系统版本号。每开始一个事务，系统版本号都会自动递增。事务开始时刻的系统版本号会作为事务的版本号，用来和查询到的每行记录的版本号进行比较。

* SELECT InnoDB会根据一下两个条件检查每行记录：
    * InnoDB只查询版本号早于当前事务的数据行，这样可以确保事务读取的行，要么是在事务开始前存在的，要么是事务本身插入或修改过的
    * 行的删除版本要么未定义，要么大于当前的事务版本号。这可以确保事务读取到行，在事务开始前未被删除
* INSERT InnoDB为新插入的每一行保存当前系统版本号作为行版本号
* DELETE InnoDB为删除的每一行保存当前系统版本号作为删除标识
* UPDATE InnoDB会插入一行新纪录，保存当前系统版本号作为行版本号，同时保存当前系统版本号到原来行作为删除标识

> **INSERT、DELETE、UPDATE操作保存的是当前系统的版本号，不是当前事务的版本号，因此解决了幻读的问题。**

由于旧数据并不真正的删除，所以必须对这些数据进行清理，InnoDB会开启一个后台线程执行清理工作，具体的规则是将删除版本号小于当前版本号的行，这个过程叫做purge

### 存储引擎

InnoDB是MySQL默认的存储引擎，也是最重要、使用最广泛的存储引擎。

* InnoDB
* MyISAM
* Archive
* Memory

选择合适的存储引擎？

大部分情况下，InnoDB是正确选择。除非需要用到某些InnoDB不具备的特性，并且没有其他方法替代，否则都应该优先选择InnoDB引擎。如果用到全文索引，建议优先考虑InnoDB加上Sphinx的组合，而不是支持全文索引的MyISAM。

**日志型应用**

Archive存储引擎

**订单处理**

InnoDB存储引擎

**InnoDB vs MyISAM**

* 事务，InnoDB支持事务，MyISAM不支持事务
* 行锁，InnoDB支持行锁，MyISAM不支持行锁
* 全文索引，InnoDB不支持全文索引，MyISAM支持全文索引
* 索引，InnoDB是聚簇索引，索引和数据放在一起，MyISAM不是聚簇索引，索引和数据是分开的
* InnoDB对于并发写的支持优于MyISAM

> 最新的版本的InnoDB存储引擎已经支持全文索引

## Schema和数据类型

### MySQL Schema设计中陷阱

* 太多的列
* 太多的关联
* 全能的枚举
* NULL的使用

**太多的列**

MySQL的存储引擎API工作时需要在服务器层和存储引擎层通过行缓存拷贝数据，然后服务器缓存内容解码成各个列。从行缓冲中将解码过的列换成行数据结构的操作代价是非常高的。如果一个非常宽，数千个字段，然后只有一小部分用到，这时的转换的代价非常高。

### 范式与反范式

范式：

* 字段不可分
* 有主健，非主键字段依赖主键
* 非主键不能相互依赖

**范式的优缺点**

* 范式更新操作比反范式要快
* 当数据较好的范式时，就只有很好或者没有重复数据，所有只需要修改更少的数据
* 范式化的表通常跟小，可以更好地放在内存里，所以执行操作更快

* 范式化设计的Schema的缺点是通常需要关联，稍微复杂的查询语句在符合范式的schema上，都可能至少一次关联

**混合范式和反范式**

最常见的反范式数据的方法是复制或缓存，在不同的表中存储相同的特定列。在MySQL5.0之后的版本，可以使用触发器更新缓存，这使得实现这样的方案变得更简单。

## 高性能索引

* *https://www.cnblogs.com/winner192/p/11747509.html*

索引（在MySQL中也叫做健key）使存储引擎用于快速找到记录的一种数据结构。

索引对于良好的性能非常关键。尤其是当表的数据结构越来越大时，索引对性能的影响愈发重要。当数据量较小且负载很低时，不恰当的索引对性能的影响可能还不明显，但当数据量逐渐增大时，性能则会急剧下降。

索引优化应该是对查询性能优化最有效的手段了。索引能够轻易将查询性能提高几个数量级，最优的索引有时比一个好的索引要好两个数量级。

### 索引的类型

* B-Tree索引
* Hash索引
* 全文索引

#### B-Tree索引

B-Tree索引能够加快访问数据的速度，因为存储不再需要进行全表扫描获取需要的数据，取而代之时从索引的节点开始搜索。根节点的槽中存放了指向子节点的指针，存储引擎根据这些指针向下查找。通过比较节点的值和要查找的值可以找到合适的指针进入下层子节点，这些指针实际上定义了子节点值的上限和下限。最终存储引擎上要找到对应的值，要么记录不存在。

叶子节点比较特殊，他们的指针指向的是索引的数据，而不是其他节点。

**可以使用B-Tree索引的查询类型**

* 全值匹配
* 匹配最左前缀
* 匹配列前缀
* 匹配范围值
* 只访问索引的查询

#### 哈希索引

哈希索引基于哈希表实现，只有精确匹配索引所有列的查询才有效。

MySQL中只有Memory存储引擎显式支持哈希索引。

#### 全文索引

全文索引是一种特殊类型的索引，它的查找的是文本中关键字，而不是直接比较索引中的值。全文索引更类似于搜索引擎做的事情，而不是简单的WHERE条件查询。

### 索引的优点

* 索引大大减少了服务器需要扫描的数据量
* 索引可以帮助服务器避免排序和临时表
* 索引可以见随机I/O变成顺序I/O

### 高性能的索引策略

正确的创建和使用索引是实现高性能索引查询的基础。

#### 独立的列

```sql
SELECT actor_id FROM sakila.actor WHERE actor_id+1=5;
```

> 正确的方式应该是actor_id=4

#### 多列索引

很多人对多列索引的理解都不够。一个常见的错误是，为每列创建独立的索引，或者按照错误的顺序创建多列索引。

```sql
SELECT film_id, actor_id FROM sakila.film_actor WHERE actor_id=1 OR film_id=1;
```

优化
```sql
SELECT film_id, actor_id FROM sakila.film_actor WHERE actor_id=1
UNION ALL
SELECT film_id, actor_id FROM sakila.film_actor WHERE film_id=1 AND actor_id<>1;
```

> 可以使用`actor_id`和`film_id`索引

#### 选择合适的索引列顺序

正确的顺序依赖于使用该索引的查询，并且同时需要考虑如何更好的满足排序和分组的需要。

如何选择索引的列的顺序？

将选择性最高的列放到索引的最前列。

#### 聚簇索引

**聚簇索引并不是一种单独的索引类型，而是一种数据存储方式。** 具体的细节依赖于具体实现方式，但InnoDB的聚簇索引实际上在同一个结构中保存了B-Tree索引和数据行。

当表有聚簇索引时，它的数据行实际上存放在索引的叶子中。术语“聚簇”表示数据行和相邻的健值紧凑的存储在一起。因为无法同时把数据行存放在两个不同的地方，所以一个表只能有一个聚簇索引。

聚集数据的优点

聚簇表示数据行和相邻的键值紧凑地存储在一起。

* 可以把相关的数据保存在一起
* 数据访问更快
* 使用覆盖索引扫描的查询可以直接使用页节点的主键值

缺点

* 聚簇数据最大限度的提高I/O密集型的应用，但如果把全部数据都放在内存中，则访问顺序就没那么重要了
* 插入速度严重依赖于插入顺序
* 更新聚簇索引的代价很高，因为会强制InnoDB将每个被更新的行移动到新的位置
* 聚簇索引可能导致全表扫描变慢，尤其是行比较稀疏，由于页分裂导致数据存储不连续
* 二级索引可能比想象的要大，因为二级索引大叶子节点包含了引用行的主键列

#### 覆盖索隐

* *https://blog.csdn.net/qq_15037231/article/details/87891683*

通常大家都会根据查询的WHERE条件创建合适的索引，不过这只是索引优化的一个方面。索引确实是一个查找数据的高效方式，但是MySQL也可以使用索引来直接获取列的数据，这样就不需要读取数据行。如果索引的叶子节点中已经包含要查询的数据，那么还有什么必要再回表查询呢？如果一个索引包含所有需要查询的字段值，我们就称之为**覆盖索引**。

* 索引条目通常远小于数据行的大小，所以如何只需要读取索引，那么MySQL就会极大地减少数据访问量。覆盖索引对于I/O密集型的应用也有帮助，因为索引比数据更小，更容易全部放入内存中。
* 一些存储引擎如MyISAM在内存中只缓存索引，数据则依赖于操作系统来缓存，因此要访问数据需要一次系统调用。这可能会导致严重的性能问题，尤其是那些系统调用占了数据访问中的最大开销的场景。
* 由于InnoDB的聚簇索引，覆盖索引对InnoDB表特别有用。InnoDB的索引在叶子节点保存了行主键，所以如何二级主键能够覆盖查询，则可以避免对主键的二次查询。

MySQL只能使用B-Tree索引做覆盖索引。

MySQL查询优化器会在执行查询前判断是否有一个索引能进行覆盖。假设索引覆盖了WHERE条件的字段，但不是整个查询涉及对字段。如果条件为false，MySQL5.5或更早的版本总是会回表获取数据行。

> Explain语句的Extra列的“Using index”和type列的“index”两者完全不同，type列和覆盖索引毫无关系。type列表示这个查询访问数据对方式，或者说是MySQL查询行的方式。

```sql
SELECT * FROM products WHERE actor="SEAN CARREY";
```

优化

```sql
SELECT * FROM products
    JOIN (SELECT prod_id FROM products WHERE actor='SEAM CARREY'
         ) as t1 ON t1.prod_id=products.prod_id;
```

我们把这种方式叫做延迟关联(deferred join)，因为延迟了对列的访问。

#### 使用索引扫描来做排序

MySQL有两种方式可以生成有序结果：通过排序操作，或者按索引扫描。如果EXPLAIN出来的type列的值为index，则说明MySQL使用了索引扫描来做排序（不要和Extra列的"using index"搞混了）。

MySQL可以使用同一个索引既能满足排序，又用于查询行。因此，如果可能，设计索引时应该尽量可能同时满足这两项任务。

> 只有索引的顺序和ORDER BY子句的顺序完全一致，并且所有列的排序方向都一样是，MySQL才能使用索引来对结果做排序。

#### 冗余重复索引

MySQL允许在相同的列上创建多个索引，无论是有意还是无意。MySQL需要单独维护重复的索引，并且优化器查询的时候也需要逐个地进行考虑，这会影响性能。

重复索引是指在相同的列上按照相同的顺序创建的相同类型的索引。应该避免这样创建重复索引，发现以后立即删除。

如果创建了（A，B）索引，再创建（A）索引就是冗余索引，因为这只是前一个索引的前缀索引。因此索引（A，B）也可以当作索引（A）来使用。但是如果再创建（B，A）索引，则不是冗余索引，索引（B）也不是，因为B不是索引（A，B）的最前缀索引。另外，其他类型的索引，例如哈希索引，也不是B-Tree索引。

冗余索引常发生在为表新添加索引的时候。应该是尽量扩展已有的索引而不是创建新索引。

另外，表的索引越多，会导致INSERT、UPDATE、DELETE等操作的速度变慢。特别是新增索引后导致内存瓶颈的时候。

#### 未使用的索引

除了重复索引，可能还存在一些服务器永远不用的索引。这样的索引完全是累赘，建议直接删除。

#### 索引和锁

*高性能MysQL P181*

#### 总结

在MySQL中，大多数情况下，都是用B-Tree索引。

索引查询的原则：

* 按顺序访问范围数据是很快的。
    * 顺序I/O不需要多次磁盘寻址
    * 如果服务器能按顺序读取数据，那么就不再需要额外的排序操作，并且GROUP BY查询也无须再做排序和将行进行聚合计算了
* 索引覆盖查询是很快的。如果一个索引包含了查询需要的所有列，那么存储引擎就可以不再需要再回表查找行了。

**如何判断一个系统创建的索引是否合理呢？**

一般来说，建议按响应时间来对查询进行分析：

* 找出那些消耗最长时间的查询或着那些给服务器带来很大压力的查询
* 然后检查这些查询的schema、SQL和索引结构，判断是否查询了扫描了太多的行
* 是否做了很多额外的排序或者使用了临时表
* 是否使用了随机I/O访问数据
* 或者是太多回表查询那些不在索引中列的操作。

既查询时间、索引结构、外排序、临时表、随机I/O、回表查询

> 查询了太多的字段，需要回表。

## 查询优化

查询优化、索引优化、库表优化。

### 慢查询基础：优化数据访问

大部分性能低下的查询都可以通过减少访问的数据量的方式进行：

* 确认应用程序是否检索大量超过需要的数据
* 确认MySQL服务器是否在分析大量超过需求的数据行

#### 是否向数据库请求了不需要的数据

* 查询不需要的记录
* 多表关联查询时返回全部列
* 总是取出全部列
* 重复查询相同的数据

#### MySQL是否在扫描额外的记录

* 响应时间
* 扫描的行数和放回的行数
* 扫描的行数和访问的类型


### 重构查询的方式

#### 一个复杂查询还是多个简单查询

#### 切分查询

#### 分解关联查询

* 让缓存更有效率
* 将查询分解后，执行单个查询可以减少锁的竞争
* 在应用层作关联，可以更容易对数据进行拆分，更容易做大高性能和可扩展
* 查询本身效率也可能会有所提升
* 可以减少冗余查询

### 查询执行的基础

![](https://ss3.bdstatic.com/70cFv8Sh_Q1YnxGkpoWK1HF6hhy/it/u=895948001,3848009024&fm=26&gp=0.jpg)

1. 客户端/服务端通信协议
2. 查询缓存
3. 查询优化处理
4. 查询执行引擎
5. 返回结果给客户端

#### MySQL如何执行关联查询

MySQL认为任何一个查询都是一次“关联”，并不仅仅是一个查询需要两个匹配才叫关联，所以MySQL中，每一个查询，每一个片段（包括子查询，甚至单表的SELECT）都可能是关联。所以，理解关联查询至关重要。

UNION查询，MySQL先将一系列查询的单个查询结果放到一个临时表中，然后再重新读出临时表的数据完成UNION查询。

### MySQL查询优化器的局限性

#### 关联子查询

**使用左外连接代替子查询**

#### Union的限制

### 优化特定类型的查询

#### 优化COUNT()查询

COUNT()是一个特殊的函数，它可以统计某个列的数量，也可以统计行数。

> COUNT(*) *并不会像我们猜想的那样扩展成所有列，实际上，它会忽略所有列而直接统计所有行的行数。

#### 优化关联查询

* 确保ON或USING子句中的列上有索引
* 确保任何GROUP BY 和 ORDER BY中的表达式只涉及到一个表中的列，这样MySQL才能有可能使用索引来优化这个过程
* 当升级MySQL的时候需要注意，关联语法，运算符优先级

#### 优化子查询

优化子查询就是尽可能使用关联查询。

#### 优化GROUP BY和DISTINCT

在MySQL中，当无法使用索引的时候，GROUP BY使用两种策略来完成：使用临时表或者文件排序来做分组。

#### 优化LIMIT分页

在系统中需要分页操作的时候，通常会使用LIMIT加上偏移量的办法来实现，同时加上合适的ORDER BY子句。如果对应的索引，通常效率会不错，否则，MySQL需要做大量的文件排序操作。

```sql
SELECT film_id, description FROM sakila.film ORDER BY title LIMIT 50 5;
```

优化后


```sql
SELECT film.film_id, film.description FROM sakila.film 
    INNER JOIN ( SELECT film_id FROM sakila.film ORDER BY title LIMIT 50, 5)
    AS lim USING(film_id);
```

> 延迟关联将大大提升效率，它让MySQL扫描尽可能少的页面，获取需要访问的记录再根据关联列取回表需要的所有列。

#### 优化UNION查询

MySQL总是通常创建并填充临时表的方式来执行UNION查询。

**除非确实是需要服务器消除重复行，否则一定要使用UNION ALL**，这一点非常重要。如果没有ALL关键字，MysQL会给临时表加上DISTINCT选项，这会导致对整个临时表的数据做唯一性检查。这样做的代码非常高。即使有ALL关键字，MySQL仍然会使用临时表存储结果。事实上，MySQL总是将结果放入临时表，然后读出，再返回给客户端。

## EXPLAIN

EXPLAIN命令是查看查询优化器如何决定执行查询的主要方法。

EXPLAIN只能解释SELECT查询，并不会对存储过程程序调用和INSERT、UPDATE、DELETE或其他语句做解释。

### EXPLAIN中的列

#### ID列

这一列总是一个编号，标识SELECT所属的行。

**MySQL将SELECT查询分为简单和复杂类型，复杂类型可以分为三大类：简单子查询、所谓派生表（FROM子句中的子查询），以及UNION查询。**

> FROM字句的子查询是派生表，但派生表不一定是FROM字句的子查询，派生表在SQL中含义很广泛。

### select_type列

这一列显示了对应行是简单还是复杂SELECT。**SIMPLE值意味着查询不包括子查询和UNION。如果查询有任何复杂的子部分，则最外层的部分标记为PRIMARY，其他部分标记如何下：**

* SUBQUERY 包含在SELECT列表中的子查询的SELECT标记为SUBQUERY
* DERIVED 表示在FROM子句的子查询的SELECT，MySQL会递归执行并将结果放到临时表中，服务器内部也称其为“派生表”，因为临时表是子查询中派生来的
* UNION 在UNION中的第二个和随后的SELECT被标记为UNOIN
* UNION RESULT 用来从UNION的匿名临时表检索结果的SELECT被标记为UNION RESULT

SUBQUERY和UNION还可以标记为DEPEDENT和UNCACHEABLE。DEPENDENT以为SELECT依赖于外层查询中发现的数据；UNCANCHEBLE意味着SELECT中某些特性组织结果被缓存一个Item_cache中。

#### table列

这一列显示了对应行正在访问哪个表。

#### type列

访问类型。

* ALL
* index
* range
* ref
* eq_ref
* const, system
* NULL

**ALL**

这就是人们常称的全表扫描，通常以为这MySQL必须扫描整张表

**index**

这个跟全表扫描一样，只是MySQL扫描表是按索引次序进行而不是行。它的主要优点就是避免了排序，最大的缺点就是要承担按索引次序读取整个表的开销。

**如果Extra列中看到“Using index”，说明MySQL正在使用覆盖索引**，它只扫描索引数据，而不是按索引次序的没一行。它比按索引次序全表扫描的开销要少得多。

**range**

范围扫描是一个有限制的索引扫描，它开始索引里的某一点，返回匹配这个值域的行。这比全索引扫描要好一些，因为不用遍历全部索引。范围扫描是带有BETWEEN或WHERE子句带有>查询。

**ref**

这是一种索引访问（有时也叫索引查找），它返回所有匹配某个单个值的行。把它叫做ref是因为索引要跟某个参考值比较。这个参考值或者是一个常数，或者来自多表前一个表里的结果值。

**eq_ref**

使用这种索引查找，MysQL知道最多只返回一条符合条件的记录。

**const, system**

当MySQL能对查询的某个部分进行优化并将其转换成一个常量时，它就会这些访问类型。

**NULL**

这种访问类型意味着MySQL能在优化阶段分解查询语句，在执行阶段甚至用不着再访问表或索引。

#### key列

这一列显示了MySQL决定采用哪个索引来优化该表的访问。

#### key_len列

该列显示MySQL在索引里使用字节数。

#### ref列

这一列显示了表key列记录的索引中查找值所用的列或常量。

#### rows列

这一列是MySQL估计为了找到所需要的行要读区的行数。

#### Extra列

* Using index, **表示MySQL将使用覆盖索引，以避免访问表，不要把覆盖索引和index访问混淆**
* Using where, 这意味着MySQL服务器将在存储引擎检索行后再进行过滤。许多WHERE条件里涉及索引中的列，当它读取索引时，就能被存储引擎检验。有时Using where的出现就是一个暗示：查询可受益不同的索引
* Using temporary 意味着MySQL在对查询结果排序时会使用临时表
* Using filesort 这意味着MySQL结果使用了一个外部排序，而不是按索引次序从表里读区行
* Range checked for each record， 这个值意味着没哟好用的索引，新的索引将在连接的每一行上重新估算。